In [1]:
import numpy as np
import pandas as pd
import sklearn
#import library numpy dan pandas

In [2]:
data=pd.read_csv('grabid.csv',sep=';', encoding='latin1')
#read csv file

In [3]:
## melihat seberapa banyak datanya
data.shape

(249, 2)

In [23]:
data.head(10)

,comment,kelas
0,terimakasih grab untuk promo yg nyantol di inb...,positive
1,kenapa layan grabcar sekarang gak nyaman sama ...,negative
2,daftar grab bike area bandung sudah tutup kah,netral
3,gw pake grab layan selalu muas cepat waktu dan...,positive
4,grab aplikasi eror nih ovonya ilanggg,negative
5,maaf mau tanya skrg grabfood pengamtaran nya a...,netral
6,mantap grab lagi banyak promo bulan ini terima...,positive
7,min kenapa helm helm grab bike suka pada bau j...,negative
8,min kode promo grab food ovo apa,netral
9,terimakasih min sering adain promo ya kami par...,positive


In [4]:
#ambil data secara acak sebanyak 5 data
data.sample(5)

,comment,kelas
248,tolong ya untuk para driver kalau ngtem kalau ...,netral
126,mantap grab promonya selalu ada terus setiap b...,positive
230,min apakah masih ada daftar untuk driver grab ...,netral
34,naik harga nya kok drastis banget ya saya bias...,negative
45,tambah pilih ladydriver di aplikasi nya jd unt...,positive


In [5]:
#melihat kolom yang ada
data.columns

Index(['comment', 'kelas'], dtype='object')

In [6]:
#melihat unik value dari kolom label nya
data['kelas'].value_counts()

negative    83
netral      83
positive    83
Name: kelas, dtype: int64

In [7]:
#library untuk split data test dan data training
from sklearn.model_selection import train_test_split

In [8]:
X = data['comment']

In [9]:
#label 'positve', 'negative', 'netral'
y = data['kelas']

In [10]:
#dipakai 30% untuk test size, 70% nya training size
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state=0)

In [11]:
#Text Preprocessing
import re

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer


tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9_]+' #menghilangkat username
pat2 = r'https?://[^ ]+' #menghilangkan situs website
www_pat = r'www.[^ ]+' #menhilangkan situs website

In [12]:
def proses_teks(teks):
    soup = BeautifulSoup(teks, 'lxml')
    souped = soup.get_text()
    try:
        teks = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        teks = souped
    teks_bersih= re.sub("[^a-zA-Z0-9]", " ",(re.sub(www_pat, '', re.sub(combined_pat, '', teks)).lower()))
    teks_bersih= ' '.join([word for word in teks_bersih.split() if word not in stopword_user])
    return (" ".join([x for x in tok.tokenize(teks_bersih) if len(x) > 1])).strip()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
#pakai algoritma naive bayes
from sklearn.naive_bayes import MultinomialNB

In [15]:
#Pipeline kelas perform kombinasi vectorizer dan tfidf dalam satu syntax
from sklearn.pipeline import Pipeline

In [16]:
#semua step dalam satu single step
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])

In [17]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [18]:
predictions = text_clf.predict(X_test)

In [19]:
from sklearn.metrics import confusion_matrix,classification_report

In [20]:
print(classification_report(y_test,predictions))
#statistik hasil test dari training

              precision    recall  f1-score   support

    negative       0.50      0.77      0.61        22
      netral       0.74      0.61      0.67        28
    positive       1.00      0.72      0.84        25

    accuracy                           0.69        75
   macro avg       0.75      0.70      0.70        75
weighted avg       0.76      0.69      0.71        75



In [21]:
from sklearn import metrics

In [22]:
#tingkat akurasi nya
metrics.accuracy_score(y_test,predictions)

0.6933333333333334